<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_07_4_more_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to LangChain Agents [[Video]]() [[Notebook]](t81_559_class_07_1_agents.ipynb)
* Part 7.2: Understanding LangChain Agent Tools [[Video]]() [[Notebook]](t81_559_class_07_2_tools.ipynb)
* Part 7.3: LangChain Retrival and Search Tools [[Video]]() [[Notebook]](t81_559_class_07_3_search_tools.ipynb)
* **Part 7.4: Constructing LangChain Agents** [[Video]]() [[Notebook]](t81_559_class_07_4_more_agent.ipynb)
* Part 7.5: Custom Agents [[Video]]() [[Notebook]](t81_559_class_07_5_custom_agent.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental duckduckgo-search langchainhub sentence-transformers faiss-cpu

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.7 MB/s eta 0:00:00


# 7.4: Constructing LangChain Agents


In this chapter, we introduce a comprehensive module that showcases the capabilities of a complete LangChain agent equipped with web search, retrieval, and memory functionalities. This agent is designed to answer questions by leveraging the most up-to-date documentation about LangChain through Retrieval-Augmented Generation (RAG), surpassing the limitations of the foundational model's knowledge. By integrating real-time web search and retrieval, the agent can access and incorporate the latest information, ensuring its responses are accurate and current. Additionally, the memory component enables the agent to retain and recall pertinent information, enhancing its ability to provide informed and contextually relevant answers. Through this module, readers will learn how to create an intelligent agent that not only understands and processes queries but also continually updates its knowledge base to stay aligned with the latest advancements in LangChain.

We begin by importing the necessary libraries and modules required for our LangChain agent, including those for web search, retrieval, and memory management. Just as we have done in previous chapters, we will create an OpenAI language model (LLM) to serve as the foundation of our agent. This model will be pivotal in interpreting and generating human-like text based on the input it receives. By leveraging the OpenAI LLM, we ensure our agent has a robust and versatile language understanding capability, which will be further enhanced by integrating real-time web search and retrieval functionalities. This setup allows the agent to access and utilize the most recent and relevant information, providing comprehensive and accurate responses to user queries.


In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

#MODEL = 'gpt-4-turbo'
MODEL = 'gpt-3.5-turbo'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )


In this section, we use Retrieval-Augmented Generation (RAG) to access the latest version of the LangChain documentation, which is more current than the data the foundational model was initially trained on. We start by creating a WebBaseLoader instance to load content from the URL https://docs.smith.langchain.com/overview, ensuring we are working with the most recent documentation. Once the content is loaded, we use RecursiveCharacterTextSplitter to break the text into smaller, manageable chunks of 1000 characters each, with an overlap of 200 characters to maintain context. These chunks are then converted into embeddings using OpenAIEmbeddings(), and we index these embeddings with FAISS (Facebook AI Similarity Search) for efficient similarity searching. Finally, we create a retriever from the FAISS index, enabling our LangChain agent to query the document embeddings and retrieve the most relevant chunks of documentation in response to user questions. This approach ensures that our agent can leverage the latest information, enhancing its ability to provide accurate and up-to-date answers.

In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

We now submit a simple query to the retriever to see what data it returns to answer the quest. The returned data is from the website, not yet processed by a LLM.

In [4]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

To provide access to the web data we just downloaded, we use an agent retriever tool. This tool is created by wrapping the retriever we previously built with the VectorStoreRetrieverTool class, specifying "Documentation" as the name and a brief description to indicate its purpose. By integrating this agent retriever tool, our LangChain agent can effectively utilize the web data for generating responses. The ZeroShotAgent is then configured with this tool, and the agent is initialized with AgentExecutor. This setup empowers our agent to retrieve and process the most relevant information from the latest LangChain documentation, ensuring it can answer queries accurately and comprehensively.

In [5]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

We also demonstrate a query to the Tavily search, this will allow us to augment the downloaded data with adhoc web searches to augment both the foundation model and RAG data source.

In [6]:
search = TavilySearchResults()

search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719777546, 'localtime': '2024-06-30 12:59'}, 'current': {'last_updated_epoch': 1719776700, 'last_updated': '2024-06-30 12:45', 'temp_c': 22.2, 'temp_f': 72.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 29.99, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 55, 'cloud': 0, 'feelslike_c': 24.6, 'feelslike_f': 76.4, 'windchill_c': 17.4, 'windchill_f': 63.3, 'heatindex_c': 17.4, 'heatindex_f': 63.3, 'dewpoint_c': 11.9, 'dewpoint_f': 53.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 18.1, 'gust_kph': 29.2}}"},
 {'url': 'https://www.msn.com/en-us/weather/topstori

Next we set up the LangChain agent by integrating various tools and configuring the agent to use them effectively. We begin by defining a list of tools that includes our web search tool and the previously created retriever tool. Next, we pull a pre-defined prompt from the LangChain hub using hub.pull("hwchase17/openai-functions-agent"), which provides structured messages that guide the agent's interactions. Using this prompt, we create a tool-calling agent with the create_tool_calling_agent function, passing in our language model (LLM), the list of tools, and the prompt. Finally, we initialize the agent executor with AgentExecutor, specifying the agent and tools, and enabling verbose mode for detailed logging. This setup ensures that our LangChain agent is well-equipped to leverage both the web search and retrieval functionalities to provide accurate and up-to-date responses.

In [7]:
tools = [search, retriever_tool]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

We can see how the agent responds to a simple trivial prompt.

In [8]:
agent_executor.invoke({"input": "hi!"})





> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

Next we present it with a question about langsmith, we can see the agent utilizes several tools.

In [9]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring that you can ship quickly and with confidence. LangSmith works independently, so the use of LangChain is not necessary. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2`, `LANGCHAIN_API_KEY`, and optionally `OPENAI_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your first trace to start testing your application.\n\nLangSmith also 

We can also ask it a current events question, causing the agent to go to the search tool.

In [10]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.cbsnews.com/philadelphia/news/pa-weather-today-june-8-2024-philadelphia-jersey-shore/', 'content': "Watch CBS News Beautiful, sunny weather around Philadelphia and Jersey Shore Saturday, rain could return Sunday By Tammie Souza June 8, 2024 / 9:43 AM EDT / CBS Philadelphia PHILADELPHIA (CBS) -- High pressure is in control of the weather here in the Philadelphia area, and it will be a sunny, wonderful day with a light breeze and a high of 83 degrees in the city.  We hit 88 degrees by Thursday and could be passing 90 degrees on Friday and the heat dome over the western U.S. slides east toward us at the end of the week.  Friday could be the start of a stretch of 90-degree days in Philadelphia, even though we haven't even reached the start of astronomical summer yet - that's on June 20 at 4:51 p.m.  68 NEXT Weather Radars Hourly Forecast First

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco is currently in the low 60s in Daly City, Pacifica, and Half Moon Bay, mid-to upper 60s in South San Francisco and San Bruno, and the mid-to upper 70s in San Mateo and Redwood City. There is a threat of thunderstorms in the North Bay, East Bay, and Sacramento Valley during the morning and early afternoon, with the threat shifting northeastward to the Sierra Nevada by the evening.'}

## Adding Chat History/Memory

To enhance our LangChain agent's capabilities, we are adding chat memory to enable it to remember previous interactions and maintain context across a conversation. This addition allows the agent to provide more coherent and contextually aware responses. We start by importing necessary modules for managing chat message histories. We then create an instance of ChatMessageHistory to store the messages exchanged during the conversation. By wrapping our existing agent executor with RunnableWithMessageHistory, we integrate the chat memory functionality. This setup ensures that each session can maintain a history of messages, facilitating smoother and more natural interactions with the agent. The code snippet below illustrates this implementation:

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

We will now test the memory, I begin by introducing myself.

In [12]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Jeff"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...
Hello Jeff! How can I assist you today?

> Finished chain.


{'input': "hi! I'm Jeff",
 'chat_history': [],
 'output': 'Hello Jeff! How can I assist you today?'}

Next I ask it a question, we can see that it recalls who I am.

In [13]:
agent_with_chat_history.invoke(
    {"input": "how can LangSmith help with testing"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY

{'input': 'how can LangSmith help with testing',
 'chat_history': [HumanMessage(content="hi! I'm Jeff"),
  AIMessage(content='Hello Jeff! How can I assist you today?')],
 'output': 'LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your first trace to start testing.\n\nBy following these steps, you can leverage LangS

We can further review and test the memory.

In [14]:
agent_with_chat_history.invoke(
    {"input": "Who am I? What did I just ask you about?!"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...
You are Jeff, and you just asked me about how LangSmith can help with testing. If you have any more questions or need further assistance, feel free to let me know!

> Finished chain.


{'input': 'Who am I? What did I just ask you about?!',
 'chat_history': [HumanMessage(content="hi! I'm Jeff"),
  AIMessage(content='Hello Jeff! How can I assist you today?'),
  HumanMessage(content='how can LangSmith help with testing'),
  AIMessage(content='LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your firs